有三种计算图的构建方式：静态计算图，动态计算图，以及Autograph。

TensorFlow 2.0主要使用的是动态计算图和Autograph。

动态计算图易于调试，编码效率较高，但执行效率偏低。

静态计算图执行效率很高，但较难调试。

而Autograph机制可以将动态图转换成静态计算图，兼收执行效率和编码效率之利。

当然Autograph机制能够转换的代码并不是没有任何约束的，有一些编码规范需要遵循，否则可能会转换失败或者不符合预期。

我们将着重介绍Autograph的编码规范和Autograph转换成静态图的原理。

并介绍使用tf.Module来更好地构建Autograph。

本篇我们介绍使用Autograph的编码规范。

### 一、Autograph编码规范总结

- 被 @tf.function 修饰的函数应尽可能使用TensorFlow中的函数而不是Python中的其他函数。例如使用tf.print而不是print，使用tf.range而不是range，使用tf.constant(True)而不是True.

- 避免在@tf.function修饰的函数内部定义tf.Variable.

- 被@tf.function修饰的函数不可修改该函数外部的Python列表或字典等数据结构变量。

### 二、Autograph编码规范解析


#### 2.1 被@tf.function修饰的函数应尽量使用TensorFlow中的函数而不是Python中的其他函数。 

In [1]:
import numpy as np
import tensorflow as tf

@tf.function
def np_random():
    a = np.random.randn(3,3)
    tf.print(a)
    
@tf.function
def tf_random():
    a = tf.random.normal((3,3))
    tf.print(a)

In [5]:
# np_random每次执行都是一样的结果。
np_random()
np_random()

array([[-0.49605814,  1.00461299,  1.80199885],
       [ 0.3723527 ,  0.24311269,  0.29574776],
       [ 0.98715561,  0.44511088, -0.92468107]])
array([[-0.49605814,  1.00461299,  1.80199885],
       [ 0.3723527 ,  0.24311269,  0.29574776],
       [ 0.98715561,  0.44511088, -0.92468107]])


In [6]:
# tf_random每次执行都会有重新生成随机数。
tf_random()
tf_random()

[[2.21371579 -0.713413894 1.39831746]
 [0.0621555969 0.736083388 -1.59870958]
 [-1.29439604 1.17848086 0.306333303]]
[[2.02275705 1.39954185 -0.0872584283]
 [0.476606578 -1.33917737 -0.701817513]
 [-0.650382221 -1.67131376 0.29374516]]


#### 2.2 避免在@tf.function修饰的函数内部定义tf.Variable.

In [7]:
# 避免在@tf.function修饰的函数内部定义tf.Variable.

x = tf.Variable(1.0,dtype=tf.float32)

@tf.function
def outer_var():
    x.assign_add(1.0)
    tf.print(x)
    return(x)

outer_var() 
outer_var()

2
3


<tf.Tensor: id=32, shape=(), dtype=float32, numpy=3.0>

In [9]:
@tf.function
def inner_var():
    x = tf.Variable(1.0,dtype = tf.float32)
    x.assign_add(1.0)
    tf.print(x)
    return(x)

# 执行将报错
# inner_var()
# inner_var()

#### 2.3 被@tf.function修饰的函数不可修改该函数外部的Python列表或字典等结构类型变量。


In [12]:
tensor_list = []

# @tf.function # 加上这一行切换成Autograph结果将不符合预期！！！
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor: id=81, shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: id=82, shape=(), dtype=float32, numpy=6.0>]


In [13]:
tensor_list = []

@tf.function # 加上这一行切换成Autograph结果将不符合预期！！！
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list


append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor 'x:0' shape=() dtype=float32>]
